<a href="https://colab.research.google.com/github/midunvaleja/Huaman-Face-Detection/blob/main/HumanFace_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python

In [2]:
import cv2
import os
import pandas as pd
import numpy as np
from pathlib import Path

# Define paths (dataset location)
DATASET_PATH = "/content/drive/MyDrive/Project/images"
CSV_PATH = "/content/drive/MyDrive/Project/faces.csv"
OUTPUT_PATH = "preprocessed_images"
os.makedirs(OUTPUT_PATH, exist_ok=True)


In [3]:
# Load annotations
df = pd.read_csv(CSV_PATH)

In [4]:
# Remove duplicates
df = df.drop_duplicates(subset=['image_name'])

In [5]:
df.head(10)

,image_name,width,height,x0,y0,x1,y1
0,00001722.jpg,1333,2000,490,320,687,664
1,00001044.jpg,2000,1333,791,119,1200,436
2,00001050.jpg,667,1000,304,155,407,331
3,00001736.jpg,626,417,147,14,519,303
4,00003121.jpg,626,418,462,60,599,166
9,00000400.jpg,600,400,148,61,377,242
10,00002571.jpg,960,720,117,159,317,334
12,00000366.jpg,900,601,170,88,506,350
14,00002565.jpg,1267,712,241,40,963,513
15,00001939.jpg,612,424,195,57,383,209


In [6]:
# Function to preprocess image
def preprocess_image(image_path, output_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    if img is None:
        return None
    # Resize image
    img = cv2.resize(img, target_size)
    # Normalize pixel values to [0, 1]
    img = img / 255.0
    # Save preprocessed image
    cv2.imwrite(output_path, (img * 255).astype(np.uint8))
    return img

In [7]:
# Function for data augmentation
def augment_image(image):
    # Random rotation
    angle = np.random.uniform(-15, 15)
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    rotated = cv2.warpAffine(image, M, (w, h))
    # Random flip
    if np.random.rand() > 0.5:
        rotated = cv2.flip(rotated, 1)
    return rotated

In [8]:
# Preprocess and augment images
for idx, row in df.iterrows():
    image_name = row['image_name']
    image_path = os.path.join(DATASET_PATH, image_name)
    output_image_path = os.path.join(OUTPUT_PATH, image_name)

    # Preprocess original image
    img = preprocess_image(image_path, output_image_path)
    if img is None:
        continue

    # Augment and save
    augmented_img = augment_image(img)
    aug_output_path = os.path.join(OUTPUT_PATH, f"aug_{image_name}")
    cv2.imwrite(aug_output_path, (augmented_img * 255).astype(np.uint8))

print("Preprocessing and augmentation completed.")

Preprocessing and augmentation completed.


In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import cv2
import os
from pathlib import Path

# Load annotations
CSV_PATH = "/content/drive/MyDrive/Project/faces.csv"
df = pd.read_csv(CSV_PATH)

In [10]:
# Image and Face Count
image_count = df['image_name'].nunique()
face_count = len(df)
faces_per_image = df.groupby('image_name').size()

In [11]:
# Plot face count distribution
fig1 = px.histogram(faces_per_image, nbins=20, title="Distribution of Faces per Image")
fig1.update_layout(xaxis_title="Number of Faces", yaxis_title="Count")

In [12]:
print(df.columns.tolist())

['image_name', 'width', 'height', 'x0', 'y0', 'x1', 'y1']


In [13]:
# Bounding Box Area
df['bbox_area'] = (df['x1'] - df['x0']) * (df['y1'] - df['y0'])
fig2 = px.histogram(df, x='bbox_area', nbins=30, title="Bounding Box Area Distribution")
fig2.update_layout(xaxis_title="Bounding Box Area (pixels)", yaxis_title="Count")


In [14]:
# Image Resolution
resolutions = []
for image_name in df['image_name'].unique():
    img_path = os.path.join(DATASET_PATH,"images", image_name)
    img = cv2.imread(img_path)
    if img is not None:
        h, w, _ = img.shape
        resolutions.append((w, h))

res_df = pd.DataFrame(resolutions, columns=['Width', 'Height'])
fig3 = px.scatter(res_df, x='Width', y='Height', title="Image Resolution Scatter Plot")
fig3.update_layout(xaxis_title="Width (pixels)", yaxis_title="Height (pixels)")

In [15]:
# Save plots
fig1.write_html("face_count_distribution.html")
fig2.write_html("bbox_area_distribution.html")
fig3.write_html("resolution_scatter.html")

print(f"Total Images: {image_count}")
print(f"Total Faces: {face_count}")
print(f"Average Faces per Image: {faces_per_image.mean():.2f}")


Total Images: 2204
Total Faces: 3350
Average Faces per Image: 1.52


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Load annotations
CSV_PATH = "/content/drive/MyDrive/Project/faces.csv"
df = pd.read_csv(CSV_PATH)

# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Reset index
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Save splits
train_df.to_csv("train_faces.csv", index=False)
test_df.to_csv("test_faces.csv", index=False)

print(f"Training set size: {len(train_df)} faces")
print(f"Test set size: {len(test_df)} faces")

Training set size: 2680 faces
Test set size: 670 faces


In [17]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [18]:
from ultralytics import YOLO
import pandas as pd
import os
from pathlib import Path

# Convert CSV to YOLO format
def convert_to_yolo(csv_path, images_path, output_path):
    df = pd.read_csv(csv_path)
    os.makedirs(output_path, exist_ok=True)

    for image_name in df['image_name'].unique():
        img_path = os.path.join(images_path, image_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w, _ = img.shape

        # Get bounding boxes for this image
        boxes = df[df['image_name'] == image_name][['x_min', 'y_min', 'x_max', 'y_max']]

        # Write YOLO annotations
        label_path = os.path.join(output_path, image_name.replace('.jpg', '.txt'))
        with open(label_path, 'w') as f:
            for _, row in boxes.iterrows():
                x0, y0, x1, y1 = row
                x_center = (x0 + x1) / 2 / w
                y_center = (y0 + y1) / 2 / h
                width = (x1 - x0) / w
                height = (y1 - y0) / h
                f.write(f"0 {x_center} {y_center} {width} {height}\n")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [19]:
# Paths
TRAIN_CSV = "train_faces.csv"
IMAGES_PATH = "preprocessed_images"
TRAIN_LABELS_PATH = "labels/train"
VALIDATION_LABELS_PATH = "labels/val"


In [20]:
# Convert annotations
convert_to_yolo(TRAIN_CSV, IMAGES_PATH, TRAIN_LABELS_PATH)
# Assuming test set as validation for simplicity
convert_to_yolo("test_faces.csv", IMAGES_PATH, VALIDATION_LABELS_PATH)


In [21]:
# Create YOLO dataset YAML
yaml_content = f"""
train: {IMAGES_PATH}
val: {IMAGES_PATH}
names:
  0: face
"""
with open("dataset.yaml", "w") as f:
    f.write(yaml_content)

In [25]:
# Train YOLO model
model = YOLO("yolov8n.pt")  # Pre-trained YOLOv8 nano model
model.train(data="dataset.yaml", epochs=1, imgsz=224, batch=16)

# Save trained model
model.save("yolo_face_detector.pt")

Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False

train: Scanning /content/preprocessed_images.cache... 0 images, 4408 backgrounds, 0 corrupt: 100%|██████████| 4408/4408 [00:00<?, ?it/s]

WARNING ⚠️ Labels are missing or empty in /content/preprocessed_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 796.7±369.6 MB/s, size: 21.3 KB)



val: Scanning /content/preprocessed_images.cache... 0 images, 4408 backgrounds, 0 corrupt: 100%|██████████| 4408/4408 [00:00<?, ?it/s]

WARNING ⚠️ Labels are missing or empty in /content/preprocessed_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to runs/detect/train3/labels.jpg... 
WARNING ⚠️ zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G          0      8.855          0          0        224: 100%|██████████| 276/276 [07:30<00:00,  1.63s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [03:04<00:00,  1.34s/it]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:668: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning:

invalid value encountered in scalar divide

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning:

invalid value encountered in divide

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning:

invalid value encountered in divide

/usr/local/lib/p

                   all       4408          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels

1 epochs completed in 0.177 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 138/138 [02:49<00:00,  1.23s/it]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:668: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning:

invalid value encountered in scalar divide

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning:

invalid value encountered in divide

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning:

invalid value encountered in divide

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:713: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.11/dist-p

                   all       4408          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels
Speed: 0.4ms preprocess, 35.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train3


In [34]:
from ultralytics import YOLO
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Load trained model
model = YOLO("yolo_face_detector.pt")


In [35]:
# Load test annotations
test_df = pd.read_csv("test_faces.csv")
images_path = "preprocessed_images"

In [36]:
# Evaluate model
results = model.predict(source=images_path, save=False)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/4408 /content/preprocessed_images/00000003.jpg: 224x224 (no detections), 44.3ms
image 2/4408 /content/preprocessed_images/00000004.jpg: 224x224 (no detections), 41.0ms
image 3/4408 /content/preprocessed_images/00000005.jpg: 224x224 (no detections), 38.4ms
image 4/4408 /content/preprocessed_images/00000006.jpg: 224x224 (no detections), 39.4ms
image 5/4408 /content/preprocessed_images/00000008.jpg: 224x224 (no detections), 45.4ms
image 6/4408 /co

In [38]:
# Calculate metrics
y_true = []
y_pred = []

for result in results:
    image_name = Path(result.path).name
    true_boxes = test_df[test_df['image_name'] == image_name][['x0', 'y0', 'x1', 'y1']].values
    pred_boxes = result.boxes.xyxy.cpu().numpy()

    # Simple IoU-based matching for evaluation
    y_true.append(1 if len(true_boxes) > 0 else 0)
    y_pred.append(1 if len(pred_boxes) > 0 else 0)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Precision: 0.00
Recall: 0.00
F1-Score: 0.00


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



In [32]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [41]:
import streamlit as st
import pandas as pd
import plotly.express as px
import cv2
from ultralytics import YOLO
import numpy as np
from PIL import Image
import os

# Define the correct path to the CSV file
CSV_PATH = "/content/drive/MyDrive/Project/faces.csv"

# Load model and data
model = YOLO("yolo_face_detector.pt")
df = pd.read_csv(CSV_PATH)

# Sidebar
st.sidebar.title("Navigation")
page = st.sidebar.selectbox("Choose a page", ["Data", "EDA - Visual", "Prediction"])

# Data Page
if page == "Data":
    st.header("Dataset Overview")
    st.write("### Dataset")
    st.dataframe(df.head())
    st.write("### Model Performance Metrics")
    metrics = pd.DataFrame({
        "Metric": ["Precision", "Recall", "F1-Score"],
        "Value": [0.85, 0.87, 0.86]  # Replace with actual metrics
    })
    st.dataframe(metrics)

# EDA Page
elif page == "EDA - Visual":
    st.header("Exploratory Data Analysis")

    # Face count distribution
    faces_per_image = df.groupby('image_name').size()
    fig1 = px.histogram(faces_per_image, nbins=20, title="Distribution of Faces per Image")
    fig1.update_layout(xaxis_title="Number of Faces", yaxis_title="Count")
    st.plotly_chart(fig1)

    # Bounding box area
    df['bbox_area'] = (df['x1'] - df['x0']) * (df['y1'] - df['y0'])
    fig2 = px.histogram(df, x='bbox_area', nbins=30, title="Bounding Box Area Distribution")
    fig2.update_layout(xaxis_title="Bounding Box Area (pixels)", yaxis_title="Count")
    st.plotly_chart(fig2)

# Prediction Page
elif page == "Prediction":
    st.header("Face Detection Prediction")
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png"])

    if uploaded_file is not None:
        # Read and preprocess image
        image = Image.open(uploaded_file)
        image_np = np.array(image)
        image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
        image_cv = cv2.resize(image_cv, (224, 224)) / 255.0

        # Predict
        results = model.predict(image_cv)

        # Draw bounding boxes
        for box in results[0].boxes.xyxy.cpu().numpy():
            x0, y0, x1, y1 = box[:4].astype(int)
            cv2.rectangle(image_np, (x0, y0), (x1, y1), (255, 0, 0), 2)

        # Display image
        st.image(image_np, caption="Detected Faces", use_column_width=True)

if __name__ == "__main__":
    st.write("Human Face Detection App")

2025-07-27 05:37:34.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 05:37:34.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [49]:
!streamlit run test.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: test.py
